In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,
test_size=0.2,random_state=25)

In [4]:
len(X_train)

22

In [5]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [6]:
model = keras.Sequential([
    #kernel_initializer is the initial value of weights w1 and w2
    #bias_initializer is the initial value of bias b
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',
    kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
                  
)

model.fit(X_train_scaled,y_train,epochs=3500)

Epoch 1/3500
1/1 [==============================] - 1s 585ms/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/3500
1/1 [==============================] - 0s 4ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/3500
1/1 [==============================] - 0s 3ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/3500
1/1 [==============================] - 0s 3ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/3500
1/1 [==============================] - 0s 2ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/3500
1/1 [==============================] - 0s 3ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/3500
1/1 [==============================] - 0s 3ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/3500
1/1 [==============================] - 0s 2ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/3500
1/1 [==============================] - 0s 2ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/3500
1/1 [==============================] - 0s 3ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/3500
1/1

In [7]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 349ms/step - loss: 0.4239 - accuracy: 0.8333


[0.42385685443878174, 0.8333333134651184]

In [8]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [9]:
model.predict(X_test_scaled)

array([[0.684597  ],
       [0.43187404],
       [0.2283642 ],
       [0.521881  ],
       [0.7000083 ],
       [0.7827023 ]], dtype=float32)

In [10]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [11]:
#to get final weights and bias after training of neural network got complete

coef,intercept = model.get_weights()
coef,intercept

(array([[3.6178918],
        [1.2327931]], dtype=float32),
 array([-2.158223], dtype=float32))

In [12]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [13]:
#implementing neural network in python fro scratch without using tf

def prediction_function(age,affordability):
    weighted_sum = coef[0]*age + coef[1]*affordability + intercept
    return sigmoid(weighted_sum)
    

In [14]:
prediction_function(0.47,1)

0.6845970421484285

In [15]:
prediction_function(0.18,1)

0.431874020887696

In [16]:
#to implement gradient descent from scratch

def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [ max(i,epsilon) for i in y_predicted]
    y_predicted_new = [ min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))
    

In [17]:
# a numpy sigmoid of vectors

def sigmoid_numpy(X): 
    return 1/(1+ np.exp(-X))

    
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

### Now implementing multilayered neural network using custom Model

In [21]:
# gradient descent function 
#to find w1,w2,bias

def gradient_descent(age,affordability,y_true,epochs,loss_threshold):
    
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    
    n = len(age)
    
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        
        loss = log_loss(y_true,y_predicted)
        
        #derivative of w1,w2 & bias
        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted - y_true))
        bias_d = np.mean(y_predicted - y_true)
        
        #new values 
        w1 = w1 - rate*w1d
        w2 = w2 - rate*w2d
        bias = bias - rate*bias_d
        
        print( f'Epoch:{i} , w1:{w1} , w2:{w2} , bias: {bias} , loss:{loss}')
        
        if (loss <= loss_threshold):
            break
        
    return w1,w2,bias
        


In [22]:
# want to stop when loss threshold is 0.4985 for proper comparison of weights with that of tf
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4985)

Epoch:0 , w1:0.974907633470177 , w2:0.948348125394529 , bias: -0.11341867736368583 , loss:0.7113403233723417
Epoch:1 , w1:0.9556229728273669 , w2:0.9058873696677865 , bias: -0.2122349122718517 , loss:0.681264778737757
Epoch:2 , w1:0.9416488476693794 , w2:0.8719790823960313 , bias: -0.2977578997796538 , loss:0.6591474252715025
Epoch:3 , w1:0.9323916996249162 , w2:0.8457541517722915 , bias: -0.3715094724003511 , loss:0.6431523291301917
Epoch:4 , w1:0.9272267472726993 , w2:0.8262362885332687 , bias: -0.43506643026891584 , loss:0.6316873063379158
Epoch:5 , w1:0.9255469396815343 , w2:0.8124402814952774 , bias: -0.48994490058938817 , loss:0.623471707997592
Epoch:6 , w1:0.9267936114129968 , w2:0.8034375029757677 , bias: -0.5375299543522853 , loss:0.6175321183044205
Epoch:7 , w1:0.93047170420295 , w2:0.7983920007454487 , bias: -0.5790424270894963 , loss:0.6131591858705934
Epoch:8 , w1:0.9361540784567942 , w2:0.7965748796787705 , bias: -0.6155315088627655 , loss:0.6098518179750948
Epoch:9 , w1:

(3.5217073640113554, 1.4075363384987245, -2.2920916336449575)

In [24]:
coef,intercept

(array([[3.6178918],
        [1.2327931]], dtype=float32),
 array([-2.158223], dtype=float32))

In [ ]:
customModel = myNN()
customModel.fit(X_train_scaled,y_train,epochs = 3000, loss_threshold = 0.4985)
    

In [ ]:
customModel.predict(X_test_scaled)